## Imports and Setup

In [1]:
import os
import sys
sys.path.insert(0, "../utils/")
import geoutils
import cnn_utils
import pred_utils
import config

import pandas as pd
import numpy as np
import geopandas as gpd
from huggingface_hub import PyTorchModelHubMixin

import torch
import torchsummary
import logging

pd.set_option('mode.chained_assignment', None)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%load_ext autoreload
%autoreload 2

C:\Users\Issa\miniconda3\envs\GFDRR\lib\site-packages\groundingdino\models\GroundingDINO\ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")


## Load Configs: Roof Type

In [24]:
prefix = '.'
phases = ["TRAIN", "TEST"]
config_file = "../configs/cnn/cnn-roof_type-efficientnetb0-RGB_DOM.yaml"
dom_c = config.load_config(config_file, prefix=prefix)
print(f"Config: {dom_c}")
dom_data, dom_data_loader, dom_classes = cnn_utils.load_dataset(dom_c, phases, prefix)

INFO:root:{'NO_ROOF': 0, 'GABLE': 1, 'HIP': 2, 'FLAT': 3}


Config: {'attribute': 'roof_type', 'data': 'RGB_DOM', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 60, 'model': 'efficientnetb0', 'pretrained': True, 'resampler': 'RandomOverSampler', 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'label_smoothing': 0.1, 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None, 'config_name': 'cnn-roof_type-efficientnetb0-RGB_DOM', 'rasters_dir': '../data/rasters/', 'vectors_dir': '../data/vectors/', 'tile_dir': '../data/tiles/', 'csv_dir': '../data/csv/', 'out_dir': '../outputs/', 'exp_dir': '../exp/', 'log_dir': '../logs/'}


In [25]:
config_file = "../configs/cnn/cnn-roof_type-resnet50-RGB_LCA.yaml"
lca_c = config.load_config(config_file, prefix=prefix)
print(f"Config: {lca_c}")
lca_data, lca_data_loader, lca_classes = cnn_utils.load_dataset(lca_c, phases, prefix)

INFO:root:{'NO_ROOF': 0, 'GABLE': 1, 'HIP': 2, 'FLAT': 3}


Config: {'attribute': 'roof_type', 'data': 'RGB_LCA', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 60, 'model': 'resnet50', 'pretrained': True, 'resampler': 'RandomOverSampler', 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'label_smoothing': 0.1, 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': 0.5, 'config_name': 'cnn-roof_type-resnet50-RGB_LCA', 'rasters_dir': '../data/rasters/', 'vectors_dir': '../data/vectors/', 'tile_dir': '../data/tiles/', 'csv_dir': '../data/csv/', 'out_dir': '../outputs/', 'exp_dir': '../exp/', 'log_dir': '../logs/'}


In [26]:
config_file = "../configs/cnn/cnn-roof_type-efficientnetb0-RGB_DOM_LCA.yaml"
com_c = config.load_config(config_file, prefix=prefix)
print(f"Config: {com_c}")
com_data, com_data_loader, com_classes = cnn_utils.load_dataset(com_c, phases, prefix)

INFO:root:{'NO_ROOF': 0, 'GABLE': 1, 'HIP': 2, 'FLAT': 3}


Config: {'attribute': 'roof_type', 'data': 'RGB_DOM_LCA', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 60, 'model': 'efficientnetb0', 'pretrained': True, 'resampler': 'RandomOverSampler', 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'label_smoothing': 0.1, 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None, 'config_name': 'cnn-roof_type-efficientnetb0-RGB_DOM_LCA', 'rasters_dir': '../data/rasters/', 'vectors_dir': '../data/vectors/', 'tile_dir': '../data/tiles/', 'csv_dir': '../data/csv/', 'out_dir': '../outputs/', 'exp_dir': '../exp/', 'log_dir': '../logs/'}


## Load Models: Roof Type

In [30]:
lca_model = pred_utils.load_model(lca_c, lca_classes)
dom_model = pred_utils.load_model(dom_c, dom_classes)
com_model = pred_utils.load_model(com_c, com_classes, push_to_hf=True)
criterion = torch.nn.CrossEntropyLoss()

INFO:root:{'model_type': 'resnet50', 'n_classes': 4, 'mode': 'RGB', 'dropout': 0.5}
INFO:root:Model file ../exp/cnn-roof_type-resnet50-RGB_LCA\cnn-roof_type-resnet50-RGB_LCA.pth successfully loaded.
INFO:root:{'model_type': 'efficientnetb0', 'n_classes': 4, 'mode': 'RGB', 'dropout': None}
INFO:root:Model file ../exp/cnn-roof_type-efficientnetb0-RGB_DOM\cnn-roof_type-efficientnetb0-RGB_DOM.pth successfully loaded.
INFO:root:{'model_type': 'efficientnetb0', 'n_classes': 4, 'mode': 'RGB', 'dropout': None}
INFO:root:Model file ../exp/cnn-roof_type-efficientnetb0-RGB_DOM_LCA\cnn-roof_type-efficientnetb0-RGB_DOM_LCA.pth successfully loaded.


In [ ]:
com_model.push_to_hub(f"issatingzon/{com_c['config_name']}")

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [31]:
from cnn_utils import GFDRRModel
model = GFDRRModel.from_pretrained(f"issatingzon/{com_c['config_name']}")

INFO:root:{'model_type': 'efficientnetb0', 'n_classes': 4, 'mode': 'RGB', 'dropout': None}


Loading weights from local directory


## Generate Results: Roof Type

In [ ]:
test_results, test_cm = cnn_utils.evaluate(
    dom_data_loader["TEST"], 
    dom_classes, 
    com_model, 
    criterion, 
    device=device,
    logging=logging
)

 59%|████████████████████████████████████████████████▎                                 | 33/56 [02:38<01:31,  3.99s/it]

In [12]:
test_results, test_cm = cnn_utils.evaluate(
    dom_data_loader["TEST"], 
    dom_classes, 
    com_model, 
    criterion, 
    device=device,
    logging=logging
)

100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [01:31<00:00,  1.63s/it]
INFO:root:Val Loss: 12.651954744543348 {'overall_accuracy': 87.38839285714286, 'balanced_accuracy': 88.01062436126446, 'f1_score_micro': 87.38839285714286, 'f1_score': 88.01571234605231, 'precision_score': 88.51167631057467, 'recall_score': 88.01062436126446, 'f1_per_class': array([91.69550173, 87.87632221, 89.93288591, 82.55813953]), 'precision_per_class': array([92.98245614, 92.46575342, 92.79778393, 75.80071174]), 'recall_per_class': array([90.44368601, 83.72093023, 87.23958333, 90.63829787]), 'loss': 12.651954744543348}


In [13]:
test_results, test_cm = cnn_utils.evaluate(
    lca_data_loader["TEST"], 
    lca_classes, 
    com_model, 
    criterion, 
    device=device,
    logging=logging
)

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [01:00<00:00,  1.96s/it]
INFO:root:Val Loss: 5.9891297817230225 {'overall_accuracy': 95.66532258064517, 'balanced_accuracy': 90.35619988626395, 'f1_score_micro': 95.66532258064517, 'f1_score': 91.9153002027303, 'precision_score': 93.63181188914643, 'recall_score': 90.35619988626395, 'f1_per_class': array([87.75510204, 97.17707442, 97.8805395 , 84.84848485]), 'precision_per_class': array([91.4893617 , 95.7841484 , 98.83268482, 88.42105263]), 'recall_per_class': array([84.31372549, 98.61111111, 96.94656489, 81.55339806]), 'loss': 5.9891297817230225}


In [6]:
test_results, test_cm = cnn_utils.evaluate(
    dom_data_loader["TEST"], 
    dom_classes, 
    lca_model, 
    criterion, 
    device=device,
    logging=logging
)

100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [03:52<00:00,  4.15s/it]
INFO:root:Val Loss: 16.294595309666224 {'overall_accuracy': 82.31026785714286, 'balanced_accuracy': 81.39380528491141, 'f1_score_micro': 82.31026785714286, 'f1_score': 82.59474845208263, 'precision_score': 84.72277098268908, 'recall_score': 81.39380528491141, 'f1_per_class': array([85.2398524 , 82.84697509, 86.04651163, 76.24565469]), 'precision_per_class': array([92.4       , 76.07843137, 86.26943005, 84.14322251]), 'recall_per_class': array([79.10958904, 90.9375    , 85.82474227, 69.70338983]), 'loss': 16.294595309666224}


In [7]:
test_results, test_cm = cnn_utils.evaluate(
    lca_data_loader["TEST"], 
    lca_classes, 
    dom_model, 
    criterion, 
    device=device,
    logging=logging
)

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [01:10<00:00,  2.28s/it]
INFO:root:Val Loss: 7.619411330069265 {'overall_accuracy': 93.04435483870968, 'balanced_accuracy': 87.92150614511655, 'f1_score_micro': 93.04435483870968, 'f1_score': 88.39089483448461, 'precision_score': 88.92682726496521, 'recall_score': 87.92150614511655, 'f1_per_class': array([81.25      , 95.25452977, 94.59459459, 82.46445498]), 'precision_per_class': array([82.9787234 , 94.19795222, 97.22222222, 81.30841121]), 'recall_per_class': array([79.59183673, 96.33507853, 92.10526316, 83.65384615]), 'loss': 7.619411330069265}


## Load Configs: Roof Material

In [10]:
prefix = '.'
phases = ["TRAIN", "TEST"]
config_file = "../configs/cnn/cnn-roof_material-resnet50-RGB_DOM.yaml"
dom_c = config.load_config(config_file, prefix=prefix)
print(f"Config: {dom_c}")
dom_data, dom_data_loader, dom_classes = cnn_utils.load_dataset(dom_c, phases, prefix)

INFO:root:{'INCOMPLETE': 0, 'HEALTHY_METAL': 1, 'IRREGULAR_METAL': 2, 'CONCRETE_CEMENT': 3, 'BLUE_TARP': 4}


Config: {'attribute': 'roof_material', 'data': 'RGB_DOM', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 60, 'model': 'resnet50', 'resampler': 'RandomOverSampler', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'label_smoothing': 0.1, 'lr': 1e-05, 'img_size': 224, 'patience': 7, 'dropout': 0.5, 'momentum': None, 'gamma': None, 'step_size': None, 'config_name': 'cnn-roof_material-resnet50-RGB_DOM', 'rasters_dir': '../data/rasters/', 'vectors_dir': '../data/vectors/', 'tile_dir': '../data/tiles/', 'csv_dir': '../data/csv/', 'out_dir': '../outputs/', 'exp_dir': '../exp/', 'log_dir': '../logs/'}


In [11]:
config_file = "../configs/cnn/cnn-roof_material-efficientnetb0-RGB_LCA.yaml"
lca_c = config.load_config(config_file, prefix=prefix)
print(f"Config: {lca_c}")
lca_data, lca_data_loader, lca_classes = cnn_utils.load_dataset(lca_c, phases, prefix)

INFO:root:{'INCOMPLETE': 0, 'HEALTHY_METAL': 1, 'IRREGULAR_METAL': 2, 'CONCRETE_CEMENT': 3}


Config: {'data': 'RGB_LCA', 'mode': 'RGB', 'attribute': 'roof_material', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 60, 'model': 'efficientnetb0', 'pretrained': True, 'resampler': 'RandomOverSampler', 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'label_smoothing': 0.1, 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None, 'config_name': 'cnn-roof_material-efficientnetb0-RGB_LCA', 'rasters_dir': '../data/rasters/', 'vectors_dir': '../data/vectors/', 'tile_dir': '../data/tiles/', 'csv_dir': '../data/csv/', 'out_dir': '../outputs/', 'exp_dir': '../exp/', 'log_dir': '../logs/'}


In [12]:
config_file = "../configs/cnn/cnn-roof_material-inceptionv3-RGB_DOM_LCA.yaml"
com_c = config.load_config(config_file, prefix=prefix)
print(f"Config: {com_c}")
com_data, com_data_loader, com_classes = cnn_utils.load_dataset(com_c, phases, prefix)

INFO:root:{'INCOMPLETE': 0, 'HEALTHY_METAL': 1, 'IRREGULAR_METAL': 2, 'CONCRETE_CEMENT': 3, 'BLUE_TARP': 4}


Config: {'attribute': 'roof_material', 'data': 'RGB_DOM_LCA', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 60, 'model': 'inceptionv3', 'pretrained': True, 'resampler': 'RandomOverSampler', 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'label_smoothing': 0.1, 'lr': 1e-05, 'img_size': 299, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None, 'config_name': 'cnn-roof_material-inceptionv3-RGB_DOM_LCA', 'rasters_dir': '../data/rasters/', 'vectors_dir': '../data/vectors/', 'tile_dir': '../data/tiles/', 'csv_dir': '../data/csv/', 'out_dir': '../outputs/', 'exp_dir': '../exp/', 'log_dir': '../logs/'}


## Load Models: Roof Material

In [37]:
lca_model = pred_utils.load_model(lca_c, lca_classes, push_to_hf=True)
dom_model = pred_utils.load_model(dom_c, dom_classes, push_to_hf=True)
com_model = pred_utils.load_model(com_c, com_classes)
criterion = torch.nn.CrossEntropyLoss()

INFO:root:{'model_type': 'efficientnetb0', 'n_classes': 4, 'mode': 'RGB', 'dropout': None, 'model_file': '../exp/cnn-roof_material-efficientnetb0-RGB_LCA\\cnn-roof_material-efficientnetb0-RGB_LCA.pth'}


model.safetensors:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

INFO:root:Model file ../exp/cnn-roof_material-efficientnetb0-RGB_LCA\cnn-roof_material-efficientnetb0-RGB_LCA.pth successfully loaded.
INFO:root:{'model_type': 'resnet50', 'n_classes': 5, 'mode': 'RGB', 'dropout': 0.5, 'model_file': '../exp/cnn-roof_material-resnet50-RGB_DOM\\cnn-roof_material-resnet50-RGB_DOM.pth'}


model.safetensors:   0%|          | 0.00/94.3M [00:00<?, ?B/s]

INFO:root:Model file ../exp/cnn-roof_material-resnet50-RGB_DOM\cnn-roof_material-resnet50-RGB_DOM.pth successfully loaded.
INFO:root:{'model_type': 'inceptionv3', 'n_classes': 5, 'mode': 'RGB', 'dropout': None, 'model_file': '../exp/cnn-roof_material-inceptionv3-RGB_DOM_LCA\\cnn-roof_material-inceptionv3-RGB_DOM_LCA.pth'}
INFO:root:Model file ../exp/cnn-roof_material-inceptionv3-RGB_DOM_LCA\cnn-roof_material-inceptionv3-RGB_DOM_LCA.pth successfully loaded.


In [15]:
from cnn_utils import GFDRRModel
model = GFDRRModel.from_pretrained(f"issatingzon/{lca_c['config_name']}")

INFO:root:{'model_type': 'efficientnetb0', 'n_classes': 4, 'mode': 'RGB', 'dropout': None, 'model_file': '../exp/cnn-roof_material-efficientnetb0-RGB_LCA\\cnn-roof_material-efficientnetb0-RGB_LCA.pth'}


Loading weights from local directory


## Generate Results: Roof Material

In [50]:
test_results, test_cm = cnn_utils.evaluate(
    dom_data_loader["TEST"], 
    dom_classes, 
    com_model, 
    criterion, 
    device=device,
    logging=logging
)

100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [02:39<00:00,  2.85s/it]
INFO:root:{'overall_accuracy': 87.77901785714286, 'balanced_accuracy': 87.16112127987732, 'f1_score_micro': 87.77901785714286, 'f1_score': 88.1732889593871, 'precision_score': 89.66324243659379, 'recall_score': 87.16112127987732, 'f1_per_class': array([88.96      , 88.27037773, 82.95454545, 92.35880399, 88.32271762]), 'precision_per_class': array([92.66666667, 83.77358491, 80.57395143, 94.55782313, 96.74418605]), 'recall_per_class': array([85.53846154, 93.27731092, 85.48009368, 90.25974026, 81.25      ])}


In [51]:
test_results, test_cm = cnn_utils.evaluate(
    lca_data_loader["TEST"], 
    lca_classes, 
    com_model, 
    criterion, 
    device=device,
    logging=logging
)

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [01:39<00:00,  3.19s/it]
INFO:root:{'overall_accuracy': 93.54838709677419, 'balanced_accuracy': 88.5796428872457, 'f1_score_micro': 93.54838709677419, 'f1_score': 90.37059656950322, 'precision_score': 93.10433578972257, 'recall_score': 88.5796428872457, 'f1_per_class': array([82.14285714, 95.9933222 , 90.63097514, 92.71523179]), 'precision_per_class': array([95.83333333, 94.26229508, 92.578125  , 89.74358974]), 'recall_per_class': array([71.875     , 97.78911565, 88.76404494, 95.89041096])}


In [45]:
lca_model = pred_utils.load_model(lca_c, lca_classes)
test_results, test_cm = cnn_utils.evaluate(
    dom_data_loader["TEST"], 
    dom_classes, 
    lca_model, 
    criterion, 
    device=device,
    logging=logging
)

100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [01:52<00:00,  2.01s/it]
INFO:root:{'overall_accuracy': 72.99107142857143, 'balanced_accuracy': 67.31608263730821, 'f1_score_micro': 72.99107142857143, 'f1_score': 64.1471309573423, 'precision_score': 63.138147235238584, 'recall_score': 67.31608263730821, 'f1_per_class': array([83.0449827 , 79.57371226, 69.03846154, 89.07849829,  0.        ]), 'precision_per_class': array([94.48818898, 69.13580247, 58.85245902, 93.21428571,  0.        ]), 'recall_per_class': array([74.07407407, 93.72384937, 83.48837209, 85.29411765,  0.        ])}


In [29]:
dom_model = pred_utils.load_model(dom_c, dom_classes)
test_results, test_cm = cnn_utils.evaluate(
    lca_data_loader["TEST"], 
    lca_classes, 
    dom_model, 
    criterion, 
    device=device,
    logging=logging
)

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [01:56<00:00,  3.77s/it]
C:\Users\Issa\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:2394: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
INFO:root:Val Loss: 9.074963600404802 {'overall_accuracy': 92.94354838709677, 'balanced_accuracy': 89.92859672747534, 'f1_score_micro': 92.94354838709677, 'f1_score': 90.83461005781342, 'precision_score': 73.62926547137074, 'recall_score': 89.92859672747534, 'f1_per_class': array([85.71428571,         nan, 95.57226399, 89.53488372, 92.5170068 ]), 'precision_per_class': array([91.07142857,  0.        , 94.07894737, 93.52226721, 89.47368421]), 'recall_per_class': array([80.95238095,         nan, 97.11375212, 85.87360595, 95.77464789]), 'loss': 9.074963600404802}
